In [12]:
import geopandas as gpd
from shapely.geometry import Point
from sqlalchemy.orm import sessionmaker
from config import CITY_NAME
from get_city_graph import get_city_graph
from get_city_data_from_db import get_city_data_from_db
from db_config import *



In [13]:
session = Session()

In [14]:
nodes, edges_gdf = get_city_data_from_db(session, 15)


In [16]:
print(edges_gdf)

      edge_id                                           geometry
0      561339  LINESTRING (21.21594 48.71262, 21.21583 48.71277)
1      561340  LINESTRING (21.21594 48.71262, 21.21599 48.712...
2      561341  LINESTRING (21.21692 48.72106, 21.21708 48.72115)
3      561342  LINESTRING (21.21692 48.72106, 21.21644 48.720...
4      561343  LINESTRING (21.21692 48.72106, 21.21698 48.720...
...       ...                                                ...
7421   568760  LINESTRING (21.27114 48.67143, 21.27122 48.671...
7422   568761  LINESTRING (21.27134 48.67157, 21.27124 48.67155)
7423   568762  LINESTRING (21.27134 48.67157, 21.27158 48.671...
7424   568763  LINESTRING (21.27124 48.67155, 21.27115 48.671...
7425   568764  LINESTRING (21.27124 48.67155, 21.27116 48.671...

[7426 rows x 2 columns]


In [23]:

# 1. Define Steel Aréna Parking Coordinates (in WGS84)
parking_coords = (48.719390, 21.258057)  # lat, lon
parking_point = Point(parking_coords[1], parking_coords[0])  # lon, lat

In [25]:


# 2. Ensure edges_gdf has correct CRS (EPSG:4326)
if edges_gdf.crs is None:
    edges_gdf.set_crs(epsg=4326, inplace=True)

In [26]:
# 3. Reproject both edges and point to EPSG:32634 (UTM zone 34N)
edges_projected = edges_gdf.to_crs(epsg=32634)
parking_projected = gpd.GeoSeries([parking_point], crs='EPSG:4326').to_crs(epsg=32634).iloc[0]

In [27]:
# 4. Compute distance from point to each edge
edges_projected['distance_to_parking'] = edges_projected.geometry.distance(parking_projected)

In [28]:
# 5. Find the closest edge
min_idx = edges_projected['distance_to_parking'].idxmin()
nearest_edge = edges_projected.loc[min_idx]

In [29]:
print(f"Nearest edge to Steel Arena parking is edge_id={nearest_edge['edge_id']} at {nearest_edge['distance_to_parking']:.2f} meters.")

Nearest edge to Steel Arena parking is edge_id=564114 at 98.58 meters.
